## <center> **Livrable n°1 :** </center>
### <center><i> **Classification binaire** </i></center>

‎ 
‎

Document réalisé par le **groupe n°X**, composé de :
- GAURE Warren
- Membre n°2
- Membre n°3
- Membre n°4

‎

---


### **Sommaire**

1. [Mise en contexte](#contexte)
2. [Objectif du livrable](#objectif)
3. [Importation des bibliothèques](#import)
4. [Chargement des données](#load)
5. [Exploration et visualisation des données](#exploration)
6. [Configuration de l'environnement](#configuration)
7. [Le réseau de neurones convolutif (CNN)](#cnn)
8. [Item #8](#item8)
9. [Item #9](#item9)
10. [Item #10](#item10)

‎ 

---

### 1. <a id='contexte'>Mise en contexte</a>

L’entreprise TouNum est spécialisée dans la numérisation de documents, qu’il s’agisse de textes ou d’images. Ses services sont particulièrement sollicités par des entreprises cherchant à transformer leur base documentaire papier en fichiers numériques exploitables. Aujourd’hui, TouNum souhaite aller plus loin en enrichissant son offre avec des outils basés sur le Machine Learning.

En effet, certains clients disposent d’un volume considérable de documents à numériser et expriment un besoin croissant pour des solutions de catégorisation automatique. Une telle innovation leur permettrait d’optimiser le traitement et l’exploitation de leurs données numérisées. Toutefois, TouNum ne dispose pas en interne des compétences nécessaires pour concevoir et mettre en place ces technologies.

C’est dans ce cadre que notre équipe de spécialistes en Data Science du CESI est sollicitée. Notre mission consiste à développer une première solution intégrant du captioning automatique : un système capable d’analyser des photographies et de générer une légende descriptive de manière autonome.

Heureusement, TouNum possède déjà plusieurs milliers d’images annotées, ce qui constituera une ressource précieuse pour entraîner les modèles de Machine Learning à partir d’un apprentissage supervisé.

---

### 2. <a id='objectif'>Objectif du livrable</a>

L'entreprise souhaitant automatiser la sélection de photos pour l'annotation, ce livrable fournira une méthode de classification se basant sur les réseaux de neurones, afin de filtrer les images n'étant pas des photos du jeu de données de départ. La solution s'appuyera sur l'architecture de réseau de neurones retenue à la vue des résultats obtenus.

---

### 3. <a id='import'>Importation des bibliothèques</a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

---

### 4. <a id='load'>Chargement des données</a>

Nous pouvons maintenant procéder au chargement des données, à savoir les archives contenant les images fournies par TouNum.

Pour ce faire, nous commençons d'abord par établir quelques variables importantes, comme le dossier où se trouve les images et les paramètres pour l'apprentissage.

In [ ]:
dataset_path = './dataset_livrable_1'
image_h = 180
image_w = 180
batch_s = 32

Maintenant que cela est fait, nous devons désormais partager les donnnées en deux ensembles, un destiné à l'*entraînement* du modèle et l'autre pour le *test* de ce dernier.

In [ ]:
train_set = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    labels = 'inferred',
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    image_size = (image_h, image_w),
    batch_size = batch_s
)

test_set = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    labels = 'inferred',
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    image_size = (image_h, image_w),
    batch_size = batch_s
)

La sortie indique la présence de **XXXX** fichiers au total, dont **XXXX** appartenant au jeu d'entraînement et **XXXX** au jeu de test.

---

### 5. <a id='exploration'>Exploration et visualisation des données</a>

Une fois les données réparties dans les deux ensembles, nous pouvons nous intéresser de plus près à elles en commençant d'abord par afficher le nom des labels et déterminer leur répartition.

In [ ]:
class_names = train_set.class_names
print(class_names)

In [ ]:
def label_distribution(train_set, test_set):
    class_count_train = np.zeros(len(class_names))
    class_count_test = np.zeros(len(class_names))
    
    for label in train_set.labels:
        class_count_train[label.numpy()] += 1
    
    for label in test_set.labels:
        class_count_test[label.numpy()] += 1
    
    for class_name, train_count, test_count in zip(class_names, class_count_train, class_count_test):
        class_total_count = train_count + test_count
        train_percentage = train_count / class_total_count
        test_percentage = test_count / class_total_count
        print(f'{class_name} : {class_total_count} images in total -> {train_count} for training ({train_percentage:.2f}%) and {test_count} ({test_percentage:.2f}%) for testing')

Nous observons que la répartition des données est assez similaire entre chaque label, il n'y aura donc aucun problème à ce niveau.

Nous pouvons maintenant afficher quelques images pour regarder plus en détail ce à quoi nous avons affaire.

In [ ]:
plt.figure(figsize=(8, 8))
for images, labels in train_set.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

---

### 6. <a id='configuration'>Configuration de l'environnement</a>

Nous allons maintenant configurer les données afin d'améliorer les performances lors des calculs que nous serons amenés à effectuer. Pour faire cela, nous allons utiliser deux fonctions : `Dataset.cache` et `Dataset.prefetch`. Elles nous permettent, respectivement, de maintenir les données en cache dans la mémoire et d'effectuer le prétraitement de l'élément courant du jeu de données en même temps que l'entraînement ou l'évaluation. Ces deux méthodes feront gagner du temps et réduiront de manière non-négligeable la complexité computationnelle.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_set = train_set.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_set = test_set.cache().prefetch(buffer_size=AUTOTUNE)

---

### 7. <a id='cnn'>Le réseau de neurones convolutif (CNN)</a>

Nous avons fait le choix d'utiliser une architecture basée sur les réseaux de neurones convolutifs (CNN) car ils sont parfaits pour la classification binaire d’images. Ils détectent automatiquement les motifs et les formes sans qu’on ait besoin d’extraire les caractéristiques à la main. Contrairement aux réseaux classiques (MLP), qui traitent chaque pixel séparément, les CNN tiennent compte des relations entre pixels, ce qui leur permet de mieux reconnaître les structures visuelles. En utilisant des filtres partagés, ils réduisent le nombre de paramètres à apprendre, rendant l’entraînement plus efficace et évitant le surapprentissage. Grâce au pooling, ils restent aussi performants même si un objet apparaît à un autre endroit dans l’image. En combinant ces atouts, les CNN offrent une solution puissante et optimisée pour cette tâche de classification.